A gente tem dados. E agora?

In [1]:
import pandas as pd

In [33]:
df = pd.read_csv('https://raw.githubusercontent.com/cassiasamp/calculadora-de-imoveis-jun-20/master/coleta_de_dados/dados_mercado_livre.csv')

In [68]:
df

,zona,quartos,area,preco
0,sul,1.0,45.0,950
1,sul,3.0,217.0,7000
2,sul,4.0,320.0,7000
3,sul,5.0,398.0,7000
4,sul,NaN,185.0,7000
...,...,...,...,...
1003,oeste,2.0,71.0,1300
1004,oeste,2.0,200.0,3000
1005,oeste,1.0,40.0,1200
1006,oeste,2.0,62.0,1300


Jeito mais simples de advinhar o preço (por exemplo da zona sul com 1 quarto e 40 m²):
- regressão linear
- colocar a média
- média pela área 
- metro quadrado da zona sul e multiplicar depois

In [6]:
df['preco_m2'] = df['preco']/df['area']

In [7]:
df.head()

,zona,quartos,area,preco,preco_m2
0,sul,1.0,45.0,950,21.111111
1,sul,3.0,217.0,7000,32.258065
2,sul,4.0,320.0,7000,21.875000
3,sul,5.0,398.0,7000,17.587940
4,sul,NaN,185.0,7000,37.837838


In [10]:
# area de sul, 46m², 1.0 - qual que é o preço?
df_sul = df.query('zona == "sul"')

In [12]:
df_sul['zona'].unique()

array(['sul'], dtype=object)

In [13]:
# df.query?
df_sul = df.query('zona == "sul" and quartos == 1')
# outro jeito de fazer o filtro
# df[(df['zona'] == 'sul') & (df['quartos'] == 1)]

In [15]:
df_sul.describe().T

,count,mean,std,min,25%,50%,75%,max
quartos,22.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
area,22.0,68.772727,78.793239,15.000000,30.000000,40.000000,60.000000,350.000000
preco,22.0,2575.000000,3964.297512,400.000000,662.500000,1100.000000,2100.000000,15000.000000
preco_m2,22.0,32.306477,17.752248,13.333333,21.354167,24.272727,38.035714,83.333333


In [19]:
# comandos mágicos para comparar a execução
# %timeit?
# %time?

In [25]:
media_de_preco_m2 = df_sul['preco_m2'].mean()

In [28]:
# o preço que a gente advinhou
media_de_preco_m2*50
# o preço da olx = 2.600 (tem 1000 de condominio, pode ser que tenha sido um bom chute)

1615.3238340491303

In [30]:
media_de_preco_m2*40
# o preço da olx = 4.000 (não sabemos se tem condomíno incluso)

1292.2590672393042

Modelando o problema com regressão linear

In [31]:
from sklearn.linear_model import LinearRegression

- variaveis independentes - o  que a gente vai usar para prever
- variavel dependente - o que a gente quer prever

In [35]:
df.head()

,zona,quartos,area,preco
0,sul,1.0,45.0,950
1,sul,3.0,217.0,7000
2,sul,4.0,320.0,7000
3,sul,5.0,398.0,7000
4,sul,NaN,185.0,7000


In [36]:
y = df['preco']

In [39]:
df.drop('preco', axis=1)
# columns='preco'
# X

,zona,quartos,area
0,sul,1.0,45.0
1,sul,3.0,217.0
2,sul,4.0,320.0
3,sul,5.0,398.0
4,sul,NaN,185.0
...,...,...,...
1003,oeste,2.0,71.0
1004,oeste,2.0,200.0
1005,oeste,1.0,40.0
1006,oeste,2.0,62.0


In [46]:
# sul : 0, norte : 1, leste : 2, oeste : 3 #label enconding
df_onehot = pd.get_dummies(df) # one hot enconding

In [51]:
df_onehot = df_onehot.drop('preco', axis=1)

In [52]:
X = df_onehot

In [53]:
X.head()

,quartos,area,zona_leste,zona_norte,zona_oeste,zona_sul
0,1.0,45.0,0,0,0,1
1,3.0,217.0,0,0,0,1
2,4.0,320.0,0,0,0,1
3,5.0,398.0,0,0,0,1
4,NaN,185.0,0,0,0,1


In [55]:
reg = LinearRegression()

In [57]:
y.head()

0     950
1    7000
2    7000
3    7000
4    7000
Name: preco, dtype: int64

In [59]:
X = X.fillna(X.median())

In [61]:
# regressão linear é sensível a valores faltantes
reg.fit(X, y) # ajustamos o modelo aos dados

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [63]:
# zona sul, area é 50, quartos 1 -- 1615
# zona sul, area é 40, quartos 1 -- 1292

# regressão linear é sensível a valores extremos ou outliers
reg.predict([[1, 50, 0, 0, 0, 1], [1, 40, 0, 0, 0, 1]])

array([13370.85625759, 13127.64634378])

In [65]:
# quartos, area, zona leste, norte, oeste e sul
reg.coef_

array([-7.49493437e+00,  2.43209914e+01, -3.56030043e+03, -4.73959312e+03,
        1.37551818e+01,  8.28613837e+03])

In [67]:
# onde cortamos o eixo y, constante b da equação linear
reg.intercept_

3876.163253467187

In [45]:
import numpy as np
type(np.nan)

float

In [ ]:
# Exercício: fazer o train test split do sklearn para dividir os dados entre treino e teste